In [1]:
pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [56]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset,Subset
from transformers import BertModel, AutoTokenizer
import tensorflow as tf
import torch.utils.data
import warnings

In [57]:
df1 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data/text/train.xlsx')
df2 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data/text/test.xlsx')

In [58]:
warnings.filterwarnings("ignore")

In [86]:
class classifier(nn.Module):
    def __init__(self):
        super().__init__()
        # ========================= EDIT HERE =========================
        # Define the layers of the model.
        self.fc1 = nn.Linear(768, 256)
        self.fc2 = nn.Linear(256,1 )
        # Define the activation function.
        self.activation1=nn.ReLU()
        self.activation2 = nn.Sigmoid()
        # =============================================================
        
    def forward(self, x,a):
        # ========================= EDIT HERE =========================
        # Given the input x, the function should return the output of the model.
        # Depending on the activation function, the output may vary.
        x = self.fc1(x)
        x = self.activation1(x)
        if (a==1):
          x = nn.Dropout(p=0.2)(x)
        x = self.fc2(x)
        x = self.activation2(x)
        # =============================================================
        return x

    def train(self,train_loader, criterion, optimizer):
        loss = None # loss of final epoch
        s=0
        # ========================= EDIT HERE =========================
        # Train should be done for 'num_epochs' times.
        # Define the loss function.
        # Define the optimizer.
        # Start training.
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(device)
        model = BertModel.from_pretrained('bert-base-uncased').to(device)
        for input_ids,attention_masks,token_type_ids, labels in train_loader:
          input_ids = input_ids.to(device)
          token_type_ids=token_type_ids.to(device)
          attention_masks=attention_masks.to(device)
          labels = labels.to(device)
          inputs=model(input_ids,token_type_ids,attention_masks)[1].to(device)
          optimizer.zero_grad()
          outputs = self.forward(inputs,0)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
          s=s+1 
          if (s==30):
            break
        print(f'Loss: {loss.item():.4f}')  
                  
    def evaluate(self,test_loader):
        correct = 0
        total = 0
        s=0
        predicted_labels = []
        #real_labels = []
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(device)
        model = BertModel.from_pretrained('bert-base-uncased').to(device)
        # ========================= EDIT HERE =========================
        with torch.no_grad():
            for input_ids,attention_masks,token_type_ids, labels in test_loader:
                input_ids = input_ids.to(device)
                token_type_ids=token_type_ids.to(device)
                attention_masks=attention_masks.to(device)
                labels = labels.to(device)
                inputs=model(input_ids,token_type_ids,attention_masks)[1].to(device)
                outputs = self.forward(inputs,0)
                predicted = torch.round(outputs)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                predicted_labels.append(predicted)
                #real_labels.extend(labels)
        predicted_labels = torch.cat(predicted_labels, 0)
        # =============================================================
        print(f'Accuracy: {100 * correct / total:.2f}%')
        print("Correct:", correct, "Total:", total)
        #acc = sum(predicted_labels == torch.Tensor(real_labels)).item()/5000
        #print(f'Accuracy: {acc:.4f}%')
        return predicted_labels
    def predict(self, test_loader):
        correct = 0
        total = 0
        predicted_labels = []
        #real_labels = []
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(device)
        model = BertModel.from_pretrained('bert-base-uncased').to(device)
        # ========================= EDIT HERE =========================
        with torch.no_grad():
            for input_ids,attention_masks,token_type_ids, labels in test_loader:
                input_ids = input_ids.to(device)
                attention_masks=attention_masks.to(device)
                labels = labels.to(device)
                inputs=self.model(input_ids,attention_masks)[1].to(device)
                outputs = self.forward(inputs,0)
                predicted = torch.round(outputs)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                predicted_labels.append(predicted)
                #real_labels.extend(labels)
        predicted_labels = torch.cat(predicted_labels, 0)
        # =============================================================

        return predicted_labels
    def predict_proba(self, inputs):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(device)
        inputs = inputs.to(device)
        predicted_labels = []
        #real_labels = []
        # ========================= EDIT HERE =========================
        with torch.no_grad():
                predicted_labels = self.forward(inputs,0)
                #real_labels.extend(labels)
        # =============================================================

        return predicted_labels




In [71]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased') 

train_x = df1['x']
train_y = df1['y']
encoded_texts= []
for text in train_x:
    encoded_text = tokenizer(text, return_tensors='pt', max_length=512, padding='max_length')
    encoded_texts.append(encoded_text)
label=torch.tensor(train_y,dtype=torch.float)
label = torch.unsqueeze(label, dim=1)

input_ids = torch.cat([x['input_ids'] for x in encoded_texts], dim=0)
attention_masks = torch.cat([x['attention_mask'] for x in encoded_texts], dim=0)
token_type_ids = torch.cat([x['token_type_ids'] for x in encoded_texts], dim=0)
label=torch.tensor(train_y,dtype=torch.float)
label = torch.unsqueeze(label, dim=1)
train_dataset = TensorDataset(input_ids, attention_masks,token_type_ids,label)
train_loader = DataLoader(train_dataset,batch_size=1,shuffle=True)



test_x = df2['x']
test_y = df2['y']
encoded_texts = []
for text in test_x:
    encoded_text = tokenizer(text, return_tensors='pt', max_length=512, padding='max_length')
    encoded_texts.append(encoded_text)
input_ids = torch.cat([x['input_ids'] for x in encoded_texts], dim=0)
attention_masks = torch.cat([x['attention_mask'] for x in encoded_texts], dim=0)
token_type_ids = torch.cat([x['token_type_ids'] for x in encoded_texts], dim=0)
label=torch.tensor(test_y,dtype=torch.float)
label = torch.unsqueeze(label, dim=1)
test_dataset = TensorDataset(input_ids, attention_masks,token_type_ids,label)
test_loader = DataLoader(test_dataset,batch_size=1,shuffle=False)
val_loader = DataLoader(test_dataset,batch_size=1,shuffle=True)



In [8]:
input_ids.size(),label.size()

(torch.Size([500, 512]), torch.Size([500, 1]))

In [87]:
classfication =classifier()
num_epochs=3
criterion = nn.BCELoss()
tensor = torch.zeros(500)
optimizer = optim.Adam(classfication.parameters(), lr=0.01)
for i in range(num_epochs):
  classfication.train(train_loader,criterion, optimizer)
  current=classfication.evaluate(val_loader) 
  

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loss: 0.2233


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Accuracy: 47.80%
Correct: 239 Total: 500


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

Accuracy: 93.40%
Correct: 467 Total: 500


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loss: 0.1743


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Accuracy: 94.60%
Correct: 473 Total: 500
